接下來我們將實作一個簡單的多層感知器（MLP）網路，這是第二天任務的實作部分。

MLP 是一種典型的前饋神經網路，包含輸入層、隱藏層和輸出層，並且每層之間的神經元完全連接。這個 MLP 網路將用來解決 MNIST 手寫數字分類問題。

我會分別使用 **TensorFlow** 和 **PyTorch** 來進行實作，這樣你可以在不同的框架中體驗如何構建 MLP。

### **TensorFlow 實作 MLP**
在 TensorFlow 中，我們將使用 `Sequential` 模型來構建 MLP。

#### **說明：**
- **數據預處理：** 我們從 MNIST 資料集中加載數據，並將圖像的像素值正規化到 [0, 1] 之間。
- **模型架構：** 我們使用 `Flatten` 將 28x28 的圖像展開成一維向量，然後通過一個隱藏層（128 個神經元，ReLU 激活），並使用 Dropout 防止過擬合。最後的輸出層有 10 個神經元，用來進行 0-9 的分類。
- **編譯與訓練：** 模型使用 Adam 優化器，損失函數為交叉熵（適合分類問題）。訓練模型 5 個 epochs，最後在測試集上進行評估。

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# 下載並預處理 MNIST 資料集
# MNIST 是一個手寫數字的資料集，包含 60,000 張訓練圖像和 10,000 張測試圖像
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 將圖像數據從 0-255 的範圍縮放到 0-1 之間，這樣可以加速模型的訓練
x_train, x_test = x_train / 255.0, x_test / 255.0

# 建立 MLP 模型
# 使用 Sequential 模型，這是一個線性堆疊的模型
model = models.Sequential([
    # 將 28x28 的圖像展開成一個向量，這樣可以輸入到全連接層中
    layers.Flatten(input_shape=(28, 28)),
    # 全連接層，包含 128 個神經元，使用 ReLU 激活函數
    layers.Dense(128, activation='relu'),
    # Dropout 層，隨機丟棄 20% 的神經元，防止過擬合
    layers.Dropout(0.2),
    # 輸出層，包含 10 個神經元，對應 10 個類別，使用 softmax 激活函數
    layers.Dense(10, activation='softmax')
])

# 編譯模型
# 使用 Adam 優化器，損失函數為 sparse_categorical_crossentropy，評估指標為準確率
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 訓練模型
# 使用訓練數據進行訓練，訓練 5 個 epoch
model.fit(x_train, y_train, epochs=5)

# 評估模型
# 使用測試數據進行評估，並輸出測試準確率
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


### **PyTorch 實作 MLP**

接下來，我們使用 PyTorch 來實作相同的 MLP 網路。

#### **說明：**
- **數據處理：** 我們使用 `torchvision.datasets` 來加載 MNIST 數據，並進行標準化處理。
- **模型架構：** MLP 包含兩個全連接層，第一層將 28x28 的圖像展開成一維向量，並通過 ReLU 激活函數；第二層是輸出層，使用 softmax 得到最終的分類概率。
- **訓練過程：** 使用交叉熵作為損失函數，Adam 作為優化器，訓練 5 個 epochs 並在測試集上評估準確率。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 設置資料轉換
# 將圖像轉換為張量並標準化到 [-1, 1] 範圍
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# 下載並加載 MNIST 資料集
# MNIST 是一個手寫數字的資料集，包含 60,000 張訓練圖像和 10,000 張測試圖像
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 使用 DataLoader 將資料集分批次加載
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 創建 MLP 模型
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # 將 28x28 的圖像展開成一維向量
        self.flatten = nn.Flatten()
        # 第一個全連接層，輸入為 28*28，輸出為 128
        self.fc1 = nn.Linear(28 * 28, 128)
        # Dropout 層，隨機丟棄 20% 的神經元，防止過擬合
        self.dropout = nn.Dropout(0.2)
        # 第二個全連接層，輸出為 10 個類別
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        # 前向傳播
        x = self.flatten(x)                 # 展開輸入圖像
        x = torch.relu(self.fc1(x))         # ReLU 激活函數
        x = self.dropout(x)                 # Dropout
        x = self.fc2(x)                     # 最後的輸出層
        return torch.softmax(x, dim=1)      # 使用 softmax 得到概率分佈

# 初始化模型、損失函數與優化器
model = MLP()
# 使用交叉熵損失函數
criterion = nn.CrossEntropyLoss()
# 使用 Adam 優化器，學習率為 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()  # 設置模型為訓練模式
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()           # 梯度歸零
            outputs = model(images)         # 前向傳播
            loss = criterion(outputs, labels)  # 計算損失
            loss.backward()                 # 反向傳播
            optimizer.step()                # 更新權重
            running_loss += loss.item()     # 累加損失
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")  # 輸出每個 epoch 的平均損失

# 測試模型
def test(model, test_loader):
    model.eval()  # 設置模型為評估模式
    correct = 0
    total = 0
    with torch.no_grad():  # 禁用梯度計算
        for images, labels in test_loader:
            outputs = model(images)         # 前向傳播
            _, predicted = torch.max(outputs.data, 1)  # 獲取預測結果
            total += labels.size(0)         # 累加總樣本數
            correct += (predicted == labels).sum().item()  # 累加正確預測數
    print(f'Test Accuracy: {100 * correct / total}%')  # 輸出測試準確率

# 執行訓練與測試
train(model, train_loader, criterion, optimizer, epochs=5)
test(model, test_loader)

### 小結

在這兩個實作中，我們通過 TensorFlow 和 PyTorch 建立了相同的多層感知器（MLP）模型，用於解決手寫數字的分類問題。你可以比較這兩個框架的差異，並根據你的需求選擇最適合的工具。